In [2]:
import os
import sys

sys.path.append(r'c:\users\george.crowther\documents\python\projects')

from optasense.optaframe import h5utils

from datetime import datetime
import pandas as pd
import numpy as np
import h5py as h5

import dask.dataframe as dd
from dask.delayed import delayed
from dask.diagnostics import ProgressBar

"""progress_bar = ProgressBar()
progress_bar.register()"""

'progress_bar = ProgressBar()\nprogress_bar.register()'

Get a list of all files to be converted by directory. For volumetric considerations, we will keep each stage separate and write to it's own file.

In [3]:
DESC = ''
FTYPE = '.csv'

In [4]:
raw_path = r'C:\Users\george.crowther\Dropbox (OptaSense OFS)\Rice - OptaSense Project\Production DTS Data'

dir_list = os.listdir(raw_path)
file_paths = {}

for directory in dir_list:
    for main_dir, sub_dir, filenames in os.walk(os.path.join(raw_path, directory)):
        file_paths[main_dir] = []
        for f in filenames:
            if f.startswith(DESC) and f[-4:] == FTYPE and f[-5] != ')':
                file_paths[main_dir].append(os.path.join(main_dir, f))

In [5]:
file_paths = [os.path.join(raw_path, fpath) for fpath in dir_list if fpath[-4:] == FTYPE]

In [6]:
@delayed
def read_dts_csv(f_path):
    
    def tstamp_from_f_path(f_path):
        """Helper function to get timestamp from file path"""
        tstring = f_path.split('\\')[-1]
        sep = tstring.split('_')
        day = int(sep[5])
        month = int(sep[4])
        year = int(sep[3])
        hour = int(sep[6])
        minute = int(sep[7])
        second = int(sep[8])
        
        return datetime(year = year, month = month, day = day, hour = hour, minute = minute, second = second)
    
    def get_data_start(f_path):
        with open(f_path) as f:
            line = f.readline()
            line_index = 1
            while 'Distance, Temperature' not in line:
                line = f.readline()
                line_index += 1
        return line_index +1
    
    # file_properties will contain all metadata and data from file
    file_properties = {}
    
    # get datetime from filepath
    file_properties['DTIME'] = tstamp_from_f_path(f_path)
    
    try:
        df = pd.read_csv(f_path, skiprows=13, index_col=0)
        df.columns = [file_properties['DTIME']]
    except:
        return pd.DataFrame(columns = [file_properties['DTIME']], index = [0]).T
        
    return df.T

In [7]:
edges = np.arange(0, len(file_paths))

In [17]:
for sample, edge in enumerate(edges[::500]):
    
    fpath = r'C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\{0}_DTS.dat'.format(edge)
    
    #if os.path.exists(fpath):
        #continue
    #[print(os.stat(f_path).st_size, f_path) for f_path in file_paths[edge:edge + 500]]
    dfs = [read_dts_csv(f_path) for f_path in file_paths[edge:edge + 500]]
    dfs = [df for df in dfs if df.compute() is not None]
    
    col_index = {}

    for i, frame in enumerate(dfs):
        f = frame.compute()
        if i == 0:
            col_index[str(i)] = []
            prev_columns = f.columns
            col_index[str(i)].append(f)
            current = i
        else:
            new_columns = f.columns
            if not np.array_equal(prev_columns, new_columns):
                col_index[str(i)] = []
                prev_columns = col_index[str(i)]
                col_index[str(i)].append(f)
                current = i
            else:
                col_index[str(current)].append(f)
            
    for frame in col_index.items():
        dts_frame = pd.concat(frame[1])
        if dts_frame.index[0] > pd.to_datetime('27/2/2016 15:08:00'):
            dts_frame.columns = dts_frame.columns + 38
            print("Adjusting columns")
        dts = h5utils.DTSArray(dts_frame, 'C')
        fpath = r'C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\{0}_DTS.dat'.format(edge + int(frame[0]))        
        print('Writing to {0}'.format(fpath))
        dts.write_DAT(fpath)

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\0_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\1000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\1500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\2000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\2500_DTS.

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17822_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17823_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17824_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17825_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17826_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17873_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17874_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17875_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17876_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17877_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17923_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17924_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17925_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17926_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17927_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17972_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17973_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17974_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17975_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\17976_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\28500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\29000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\29500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\30000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\30500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\53000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\53500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\54000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\54500_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\55000_DTS.dat
Adjusting columns
Writing to C:\Users\george.crowther\Dropbox (OptaSense OFS)\Customer Projects\Rice Energy\Belmont Project\data\DTS_Fracture_Treatment_02\

In [9]:
dts.data.head()

0.00,1.65,3.3,4.95,6.61,8.26,9.91,11.56,13.22,14.87,16.52,...,19036.34,19038.0,19039.65,19041.3,19042.95,19044.6,19046.26,19047.91,19049.56,19051.22
2016-04-16 10:07:39,63.21,62.92,63.34,64.03,64.40,63.70,63.22,63.71,64.42,64.74,...,154.01,150.65,115.76,20.10,-96.29,-168.69,-179.89,-178.45,-205.36,-253.70
2016-04-16 10:12:44,63.51,62.84,62.70,64.14,64.27,64.01,63.60,63.53,64.29,64.90,...,156.35,152.51,118.34,21.95,-96.63,-168.61,-180.76,-176.74,-205.92,-256.29
2016-04-16 10:17:46,63.33,62.97,63.33,63.92,64.50,64.29,63.43,63.50,64.68,64.90,...,154.03,149.24,114.82,20.84,-96.09,-167.37,-181.46,-178.40,-205.79,-257.64
2016-04-16 10:22:49,63.42,63.02,63.42,64.30,64.63,63.95,63.82,64.18,64.48,64.99,...,155.14,150.85,115.99,22.23,-96.59,-167.84,-178.92,-174.59,-206.26,-257.82
2016-04-16 10:27:51,63.66,63.43,64.02,64.57,65.23,64.64,63.91,63.99,64.93,65.47,...,155.43,151.39,117.98,24.38,-96.59,-167.87,-179.70,-177.03,-204.31,-254.03


In [ ]:
dts.data.shift